<a href="https://colab.research.google.com/github/rdkdaniel/The-Swahili-Project/blob/main/Kiswahili_Wordpiece_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kiswahili WordPiece Tokenizer**

**Introduction**

*   Kiswahili has a different strcture to English or any other well-researched language in NLP. 
*   It is therefore, important to design a tokenizer from scratch for Kiswahili to meet its NLP needs.



# **0.0 Libraries**

In [2]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 14.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.7 MB/s eta 0:00:00


In [4]:
import transformers

# **1.0 Loading the Datasets**

In [5]:
df = pd.read_csv(r'/content/drive/MyDrive/train.csv')
print(df)

            id                                            content category
0       SW4670   Bodi ya Utalii Tanzania (TTB) imesema, itafan...   uchumi
1      SW30826   PENDO FUNDISHA-MBEYA RAIS Dk. John Magufuri, ...  kitaifa
2      SW29725  Mwandishi Wetu -Singida BENKI ya NMB imetoa ms...   uchumi
3      SW20901   TIMU ya taifa ya Tanzania, Serengeti Boys jan...  michezo
4      SW12560   Na AGATHA CHARLES – DAR ES SALAAM ALIYEKUWA K...  kitaifa
...        ...                                                ...      ...
23263  SW24920   Alitoa pongezi hizo alipozindua rasmi hatua y...   uchumi
23264   SW4038   Na NORA DAMIAN-DAR ES SALAAM  TEKLA (si jina ...  kitaifa
23265  SW16649   Mkuu wa Mkoa wa Njombe, Dk Rehema Nchimbi wak...   uchumi
23266  SW23291   MABINGWA wa Ligi Kuu Soka Tanzania Bara, Simb...  michezo
23267  SW11778   WIKI iliyopita, nilianza makala haya yanayole...  kitaifa

[23268 rows x 3 columns]


# **2.0 Building The Tokenizer**

**The Process Used**

Tokenization involves 5 steps:


*   Normalization - text cleanup such as lowercasing, removing accents etc.
*   Pre-tokenization - splitting the words.
*   Model - the actual tokenization.
*   Post-processing - Special tokens are added.
*   Decoder - the final step, tokenized data converted to human-readable text.



## **2.1 Libraries**

In [6]:
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import models

In [7]:
tokenizer = Tokenizer(models.WordPiece(unk_token='[UNK]'))

## **2.2 Normalization**

In [8]:
from tokenizers import normalizers
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Lowercase(), normalizers.NFKD()]
)

## **2.3 Pre-Tokenization**

In [9]:
from tokenizers import pre_tokenizers
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace() # training from scratch or use  prebuilt BertPreTokenizer for prettained one.

## **2.4 Training the Tokenizer**

In [10]:
from tokenizers import trainers

trainer = trainers.WordPieceTrainer(
    vocab_size=30_000,
    special_tokens=['[UNK]', '[PAD]', '[CLS]', '[SEP]', '[MASK]'],
    min_frequency=2,
    continuing_subword_prefix='##'
)

In [11]:
tokenizer.train_from_iterator(df, trainer=trainer)

## **2.5 Post Processing**

In [12]:
from tokenizers import processors

# first we get the token ID values (defined in the vocab) for CLS and SEP
cls_id = tokenizer.token_to_id('[CLS]')
sep_id = tokenizer.token_to_id('[SEP]')

# then setup the post processing step with TemplateProcessing
tokenizer.post_processor = processors.TemplateProcessing(
    single=f'[CLS]:0 $A:0 [SEP]:0',
    pair=f'[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1',
    special_tokens=[
        ('[CLS]', cls_id),
        ('[SEP]', sep_id)
    ]
)

## **2.6 Decoder**

In [13]:
from tokenizers import decoders

tokenizer.decoder = decoders.WordPiece(prefix='##')

## **2.7 Saving the Tokenizer**

In [14]:
from transformers import PreTrainedTokenizerFast

# load the tokenizer in a transformers tokenizer instance
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token='[UNK]',
    pad_token='[PAD]',
    cls_token='[CLS]',
    sep_token='[SEP]',
    mask_token='[MASK]'
)

# save the tokenizer
tokenizer.save_pretrained('/content/Kisw_wordpiece_tokn')

('/content/Kisw_wordpiece_tokn/tokenizer_config.json',
 '/content/Kisw_wordpiece_tokn/special_tokens_map.json',
 '/content/Kisw_wordpiece_tokn/tokenizer.json')

# **3.0 Using the Tokenizer**

In [15]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('/content/Kisw_wordpiece_tokn')

In [16]:
tokens = tokenizer("Ilikuwa wakati wa jioni jua limepunguza udhia wake na upepo mwanana ulikuwa ukipita na kuzipapasa ngozi zetu mfano wa pamba")

In [17]:
tokens

{'input_ids': [2, 0, 0, 0, 0, 0, 0, 0, 0, 11, 21, 0, 0, 0, 0, 11, 21, 0, 0, 0, 0, 0, 0, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
print(tokenizer)

PreTrainedTokenizerFast(name_or_path='/content/Kisw_wordpiece_tokn', vocab_size=26, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [20]:
tokens.input_ids

[2, 0, 0, 0, 0, 0, 0, 0, 0, 11, 21, 0, 0, 0, 0, 11, 21, 0, 0, 0, 0, 0, 0, 3]

Ref: https://huggingface.co/course/chapter6/8?fw=pt